In [1]:
import numpy as np
from scipy.integrate import RK45,solve_ivp
from ODE_potentials import VanDerPolePotential,LotkiVolterraPotential
from ODE_samplers import MALA_ODE,ULA_ODE,grad_ascent_ODE,run_eval_test,set_function
from multiprocessing import Pool
import multiprocessing
from zv_cv import Eval_ZVCV

from baselines import construct_ESVM_kernel, construct_Tukey_Hanning 
from optimize import optimize_parallel_new
from utils import *

Parameters for van-der-Pole and Lotka-Volterra examples: 

In [2]:
typ = 'LV' #'LV' for Lotka-Volterra, 'VdP' for Van-der-Pole
method = {"sampler":"MALA"} #switch between ULA and MALA
f_type = "sum_comps"

if typ == 'VdP':
    #true parameter value
    theta_star = 1.0
    #initial coordiante and speed
    y0 = np.array([0.0,2.0],dtype=float)
    #error of measurements
    sigma = 0.5
    #prior variance
    sigma_prior = 0.5
elif typ == 'LV':
    theta_star = np.array([0.6,0.025,0.8,0.025],dtype = float)
    #initial number of victims and predators
    y0 = np.array([30.0,4.0],dtype=float)
    #setting prior parameters
    sigma_prior = np.array([0.5,0.05,0.5,0.05],dtype = float)
    mu_prior = np.array([1.0,0.05,1.0,0.05],dtype=float)
    #measurements error
    sigma = np.array([0.25,0.25])

Timestaps

In [3]:
#initial and last time moments
t0 = 0
t_bound = 10
N_steps = 10
#moments of observations
t_moments = np.linspace(t0,t_bound,N_steps+1)
print(t_moments)

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]


Creating potentials

In [4]:
if typ == 'VdP':
    Cur_pot = VanDerPolePotential(sigma,sigma_prior,t_moments,theta_star,y0,t0,t_bound)
elif typ == 'LV':
    Cur_pot = LotkiVolterraPotential(sigma,mu_prior,sigma_prior,t_moments,theta_star,y0,t0,t_bound)

system solved
[[ 37.52358801   5.44451059]
 [ 49.31015054   4.44056815]
 [ 68.02780239   7.49382876]
 [ 96.31996061  69.02188599]
 [ 34.50985353 118.75692966]
 [ 11.83786961  55.32423794]
 [  7.55525337  25.78444883]
 [  4.06696204  11.32951045]
 [ 12.3379667    3.80395742]
 [ 12.53918379   6.22803384]
 [ 18.44220932   3.86005378]]


Sampling (currently with MALA)

In [5]:
r_seed = 666
#burn-in period
N_burn = 1*10**3
#Train size
N_train = 1*10**4
#Test size
N_test = 1*10**4
#number of test trajectories
n_traj = 24
if typ == 'VdP':
    #dimension
    d = 1
    #step size
    step = 1e-3
elif typ == 'LV':
    #dimension
    d = 4
    #step size
    step = 5e-6

### Construct kernels and sample

In [12]:
# Construct kernel
#W_train_spec = construct_ESVM_kernel(N_train)
#W_test_spec = construct_ESVM_kernel(N_test)

In [13]:
if typ == 'VdP':
    params_prior = {"sigma":sigma_prior}
elif typ == 'LV':
    params_prior = {"mu":mu_prior,"sigma":sigma_prior}

### Compute starting point (maximum likelihood)

In [14]:
N_steps_ascent = 5000
traj,traj_grad = grad_ascent_ODE(1453,Cur_pot,step,params_prior,N_steps_ascent,d,typ)
theta_mle = traj[-1,:]
print("mle for parameters: ",theta_mle)
Cur_pot.set_mle(theta_mle)

mle for parameters:  [0.64131655 0.02375897 0.81483698 0.02469178]


In [15]:
#print(traj_grad[-10:,:])

In [42]:
a= [1]

def f(i,a):
    a.append(1)
    return

print(a)
n_traj = 100
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res = trav.starmap(f, [(i,a) for i in range (n_traj)])
trav.close()
print(a)

[1]
[1]


### Setting function 

In [16]:
#step_train = 5e-7
inds_arr = np.array([0])
params = {"ind":0}
t_moments = None
r_seed = 777
traj = []
traj_grad = []
#generate data
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res = trav.starmap(MALA_ODE, [(r_seed+i,Cur_pot, step, params_prior, N_burn, N_train, d, typ) for i in range (n_traj)])
trav.close()
for i in range(len(res)):
    traj.append(res[i][0])
    traj_grad.append(res[i][1])
    print("accepted = ",res[i][2])
traj = np.asarray(traj)
traj_grad = np.asarray(traj_grad)
traj_grad = (-1)*traj_grad

accepted =  8505
accepted =  8499
accepted =  8520
accepted =  8428
accepted =  8578
accepted =  8490
accepted =  8495
accepted =  8523
accepted =  8498
accepted =  8521
accepted =  8544
accepted =  8548
accepted =  8438
accepted =  8451
accepted =  8483
accepted =  8497
accepted =  8476
accepted =  8453
accepted =  8427
accepted =  8442
accepted =  8605
accepted =  8602
accepted =  8491
accepted =  8469


In [27]:
print(traj[9])

[[0.62300875 0.02720611 0.77039689 0.02667897]
 [0.61675787 0.02661809 0.76398121 0.02688849]
 [0.6078545  0.02599585 0.77303298 0.02672914]
 ...
 [0.53189131 0.01588039 0.9492699  0.03199779]
 [0.52372089 0.01541632 0.95546616 0.0323883 ]
 [0.52479701 0.01625059 0.96676459 0.03125577]]


### Apply control variates

In [ ]:
bn = int(np.sqrt(N_test))
W_test = construct_Tukey_Hanning (N_test,bn)

In [ ]:
from baselines import set_function, Spectral_var
def ZVpoly1d(traj,traj_grad,f_target,deg,lamda_reg,W_spec):
    if f_target == "sum":
        samples = traj[:,0].reshape(-1,1)
    else:
        raise "Unexpected function type in Eval_ZVCV_1d"
    L_psi = np.zeros((deg,samples.shape[0]),dtype = float)
    for i in range(deg):
        L_psi[i] = (i+1)*traj_grad[:,0]*(traj[:,0]**i) + i*(i+1)*(traj[:,0]**(i-1))
    #compute main matrix
    H_zv = np.cov(L_psi, rowvar=True)
    #compute right side
    b_zv = ((L_psi - L_psi.mean(axis=1).reshape(deg,1)) @ (samples - samples.mean(axis=0)))/(samples.shape[0]-1)
    theta = np.linalg.inv(H_zv + lambda_reg*np.eye(deg)) @ b_zv
    ZV_est = samples - L_psi.T @ theta
    mean_ZV = np.mean(ZV_est, axis = 0)
    var_ZV = Spectral_var(ZV_est[:,0],W_spec)
    return mean_ZV, var_ZV

def CVpoly1d(traj,traj_grad,f_target,deg,lamda_reg,W_spec):
    if f_target == "sum":
        samples = traj[:,0].reshape(-1,1)
    else:
        raise "Unexpected function type in Eval_ZVCV_1d"
    Nabla_psi = np.zeros((deg,samples.shape[0]),dtype=float)
    Psi = np.zeros((deg,samples.shape[0]),dtype=float)
    L_psi = np.zeros((deg,samples.shape[0]),dtype=float)
    for i in range(deg):
        Nabla_psi[i] = (i+1)*(traj[:,0]**i)
        Psi[i] = traj[:,0]**(i+1)
        L_psi[i] = (i+1)*traj_grad[:,0]*(traj[:,0]**i) + i*(i+1)*(traj[:,0]**(i-1))
    #compute main matrix
    H_cv = np.dot(Nabla_psi,Nabla_psi.T)/samples.shape[0]
    #compute right side
    b_cv = ((Psi - Psi.mean(axis=1).reshape(deg,1)) @ (samples - samples.mean(axis=0)))/(samples.shape[0]-1)
    theta = np.linalg.inv(H_cv + lambda_reg*np.eye(deg)) @ b_cv
    CV_est = samples - L_psi.T @ theta
    mean_CV = np.mean(CV_est, axis = 0)
    var_CV = Spectral_var(CV_est[:,0],W_spec)
    return mean_CV, var_CV

def CVpoly1d_adj(traj,traj_grad,f_target,deg,lamda_reg,W_spec):
    if f_target == "sum":
        samples = traj[:,0].reshape(-1,1)
    else:
        raise "Unexpected function type in Eval_ZVCV_1d"
    Nabla_psi = np.zeros((deg,samples.shape[0]),dtype=float)
    Psi = np.zeros((deg,samples.shape[0]),dtype=float)
    L_psi = np.zeros((deg,samples.shape[0]),dtype=float)
    for i in range(deg):
        Nabla_psi[i] = (i+1)*(traj[:,0]**i)
        Psi[i] = traj[:,0]**(i+1)
        L_psi[i] = (i+1)*traj_grad[:,0]*(traj[:,0]**i) + i*(i+1)*(traj[:,0]**(i-1))
    #compute main matrix
    Pois = np.concatenate([Psi,-L_psi],axis=0)
    Cov_matr = np.cov(Pois,rowvar = True)
    H_cv = Cov_matr[:deg,deg:]
    #compute right side
    b_cv = ((Psi - Psi.mean(axis=1).reshape(deg,1)) @ (samples - samples.mean(axis=0)))/(samples.shape[0]-1)
    theta = np.linalg.inv(H_cv + lambda_reg*np.eye(deg)) @ b_cv
    CV_est = samples + L_psi.T @ theta
    mean_CV = np.mean(CV_est, axis = 0)
    var_CV = Spectral_var(CV_est[:,0],W_spec)
    return mean_CV, var_CV
    

def Eval_ZVCV_1d(traj,traj_grad,f_target,deg,lambda_reg,W_spec):
    if f_target == "sum":
        samples = traj.sum(axis = 1).reshape(-1,1)
    else:
        raise "Unexpected function type in Eval_ZVCV_1d"
    mean_vanilla = np.mean(samples)
    vars_vanilla = Spectral_var(samples[:,0],W_spec)
    mean_ZV, var_ZV = ZVpoly1d(traj,traj_grad,f_target,deg,lambda_reg,W_spec)
    mean_CV, var_CV = CVpoly1d(traj,traj_grad,f_target,deg,lambda_reg,W_spec)
    mean_CV_pois, var_CV_pois = CVpoly1d_adj(traj,traj_grad,f_target,deg,lambda_reg,W_spec)
    return (mean_vanilla,mean_ZV, mean_CV, mean_CV_pois), (vars_vanilla, var_ZV, var_CV, var_CV_pois)

In [ ]:
#f_type = "sum"
f_type = "sum"
params = {"ind":0}
lambda_reg = 1e-10
deg = 20
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res = trav.starmap(Eval_ZVCV_1d, [(traj[i,:,:],traj_grad[i,:,:],f_type,deg,lambda_reg,W_test) for i in range (n_traj)])
trav.close()

In [ ]:
res_arr = np.asarray(res)
print(res_arr.shape)

### Save results

In [ ]:
#np.save("ode/mala_step_1e-5_obs_20_sum_15_04.npy",res_arr)
print("Average vr rates:")
print("ZV:",np.mean(res_arr[:,1,0]/res_arr[:,1,1]))
print("CV:",np.mean(res_arr[:,1,0]/res_arr[:,1,2]))
print("adjusted CV:",np.mean(res_arr[:,1,0]/res_arr[:,1,3]))

In [ ]:
#f_type = "sum"
f_type = "sum_comps"
params = {"ind":1}
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res = trav.starmap(Eval_ZVCV, [(traj[i,:,:],traj_grad[i,:,:],f_type,params,W_test) for i in range (n_traj)])
trav.close()

In [ ]:
res_arr_new = np.asarray(res)
print(res_arr_new.shape)
#np.save("ode/mala_step_1e-5_obs_20_sum_15_04.npy",res_arr)
print("Average vr rates:")
print("ZV-1:",np.mean(res_arr_new[:,1,0]/res_arr_new[:,1,1]))
print("CV-1:",np.mean(res_arr_new[:,1,0]/res_arr_new[:,1,3]))
print("ZV-2:",np.mean(res_arr_new[:,1,0]/res_arr_new[:,1,2]))
print("CV-2:",np.mean(res_arr_new[:,1,0]/res_arr_new[:,1,4]))

### Plotting the results

In [ ]:
title = ""
labels = ['Vanilla\n MALA', 'MALA \nwith EVM-1', 'MALA \nwith CV-1']

In [ ]:
data = [res_arr_new[:,0,0],res_arr_new[:,0,1],res_arr_new[:,0,3]] 
boxplot_ind(data, title, labels, path = "results/ode/mala_delta_1.pdf")

In [ ]:
title = ""
labels = ['Vanilla\n MALA', 'MALA \nwith EVM-2', 'MALA \nwith CV-2']

In [ ]:
data = [res_arr_new[:,0,0],res_arr_new[:,0,2],res_arr_new[:,0,4]] 
boxplot_ind(data, title, labels, path = "results/ode/mala_delta_2_all.pdf")

In [ ]:
title = ""
labels = ['MALA \nwith EVM-2', 'MALA \nwith CV-2']

In [ ]:
data = [res_arr_new[:,0,2],res_arr_new[:,0,4]] 
boxplot_ind(data, title, labels, path = "results/ode/mala_delta_2.pdf")